# Music dataset

In [1]:
import numpy as np
from pathlib import Path

In [2]:
DIR = Path().cwd()
DIRmusic = (DIR / ".." / "datasets" / "music").resolve()
DIR_module = DIRmusic / "music.py"
DIR_answers = DIRmusic / "answers.json"
print(DIRmusic)

/home/tlefort/Documents/peerannot/peerannot/datasets/music


# Majority vote

In [ ]:
! peerannot aggregate $DIRmusic -s MV

In [ ]:
from peerannot.runners.train import load_all_data
labels_path = DIRmusic / "labels" / "labels_music_mv.npy"
trainset, valset, testset = load_all_data(
    DIRmusic, labels_path, path_remove=None, labels=labels_path, img_size=224, data_augmentation=False)

In [ ]:
labels_path = DIRmusic / "labels" / "labels_music_mv.npy"
num_epochs = 50

In [ ]:
! peerannot train {DIRmusic} -o music_mv -K 10 \
    --labels={labels_path} --model resnet34 --img-size=224 \
    --n-epochs={num_epochs} --lr=0.01 \
    --data-augmentation \
    --num-workers 8 --optimizer sgd --pretrained --freeze -m 1000

In [ ]:
!cat /home/tlefort/Documents/peerannot/peerannot/datasets/music/results/music_mv.json

# Dawid and Skene

In [ ]:
! peerannot aggregate $DIRmusic -s DS

In [ ]:
from peerannot.runners.train import load_all_data
labels_path = DIRmusic / "labels" / "labels_music_ds.npy"
trainset, valset, testset = load_all_data(
    DIRmusic, labels_path, path_remove=None, labels=labels_path, img_size=224, data_augmentation=False)

In [ ]:
! peerannot train {DIRmusic} -o music_ds -K 10\
    --labels={labels_path} --model resnet18 --img-size=224\
    --n-epochs={num_epochs} --lr=0.001 \
    --data-augmentation \
    --num-workers 8 --optimizer adam --pretrained --freeze -m 1000

# GLAD

In [ ]:
! peerannot aggregate $DIRmusic -s glad

In [ ]:
from peerannot.runners.train import load_all_data
labels_path = DIRmusic / "labels" / "labels_music_glad.npy"
trainset, valset, testset = load_all_data(
    DIRmusic, labels_path, path_remove=None, labels=labels_path, img_size=224, data_augmentation=False)

In [ ]:
! peerannot train {DIRmusic} -o music_glad -K 10\
    --labels={labels_path} --model resnet18 --img-size=224\
    --n-epochs={num_epochs} --lr=0.001 \
    --data-augmentation \
    --num-workers 8 --optimizer adam --pretrained --freeze -m 1000

# CoNAL

In [ ]:
! peerannot aggregate-deep {DIRmusic} -o music_conal_scale \
    --answers {DIR_answers} -s conal[scale=1e-5] \
    --model=resnet18 --img-size=224 --pretrained --n-classes=10 \
    --freeze -m 10000 \
    --n-epochs={num_epochs} --lr=0.001 --optimizer=adam \
    --data-augmentation \
    --batch-size=64 --num-workers=8 --data-augmentation

In [ ]:
!cat /home/tlefort/Documents/peerannot/peerannot/datasets/music/results/music_conal_scale.json

# WAUM

In [ ]:
! peerannot identify {DIRmusic} -K 10 --method WAUM \
    --labels {DIRanswers} \
    --model resnet18 --n-epochs 250 --lr=0.001 --img-size=224 \
    --maxiter-DS=100 --pretrained --optimizer=adam --alpha=0.05 --freeze -m 10000

In [ ]:
from peerannot.runners.train import load_all_data
labels_path = DIRmusic / "labels" / "labels_waum_0.01.npy"
remove_path = DIRmusic / "identification" / "waum_0.05_yang" / "too_hard_0.01.txt"
trainset, valset, testset = load_all_data(
    DIRmusic, labels_path, path_remove=None, labels=labels_path, img_size=224, data_augmentation=False)

In [ ]:
! peerannot train {DIRmusic} -o music_waum_0.05 -K 10 \
    --labels={labels_path} --model resnet18 --n-epochs={num_epochs} \
    --lr=0.001 -m 15000 --num-workers=8 --path-remove={remove_path} \
    --data-augmentation \
    --pretrained --optimizer=adam --batch-size=64 --freeze

# CrowdLayer

In [6]:
! peerannot aggregate-deep {DIRmusic} -o music_cl_test \
    --answers {DIR_answers} -s crowdlayer[scale=0] \
    --model=resnet18 --img-size=224 --pretrained --n-classes=10 \
    --freeze -m 10000 \
    --n-epochs=2 --lr=0.001 --optimizer=adam \
    --data-augmentation \
    --batch-size=64 --num-workers=8 --data-augmentation

Running the following configuration:
----------
- Data at /home/tlefort/Documents/peerannot/peerannot/datasets/music will be saved with prefix music_cl_test
- number of classes: 10
- output_name: music_cl_test
- answers: /home/tlefort/Documents/peerannot/peerannot/datasets/music/answers.json
- strategy: crowdlayer[scale=0]
- model: resnet18
- img_size: 224
- pretrained: True
- n_classes: 10
- freeze: True
- milestones: (10000,)
- n_epochs: 2
- lr: 0.001
- optimizer: adam
- data_augmentation: True
- batch_size: 64
- num_workers: 8
- dataset: /home/tlefort/Documents/peerannot/peerannot/datasets/music
- momentum: 0.9
- decay: 0.0005
- scheduler: False
- n_params: 3072
- lr_decay: 0.1
- path_remove: None
- metadata_path: None
----------
Loading datasets
Using cache found in /home/tlefort/.cache/torch/hub/pytorch_vision_main
Using cache found in /home/tlefort/.cache/torch/hub/pytorch_vision_main
Using cache found in /home/tlefort/.cache/torch/hub/pytorch_vision_main
Successfully loaded resn

In [8]:
!cat /home/tlefort/Documents/peerannot/peerannot/datasets/music/results/music_cl_test.json

{
   "train_loss": [
      108.03543090820312,
      107.17723083496094
   ],
   "val_loss": [
      2.9743356369809564,
      3.027241955632749
   ],
   "val_accuracy": [
      9.364548494983278,
      15.719063545150501
   ],
   "test_accuracy": 9.364548494983278,
   "test_loss": 2.9743356369809564,
   "test_ece": 0.2473752796649933
}